# IBM Applied Data Science Capstone Project Week 2

## Part 1: Scraping Wikipedia to create a table

### Libraries I find useful¶

In [18]:
!pip install bs4
#!pip install requests
from bs4 import BeautifulSoup # this module helps in web scrapping
import requests # this module helps us to download a web page
import numpy as np
import pandas as pd
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from urllib.request import urlopen
html = urlopen("http://www.google.com/")
print(html)


In [19]:
# Create url and observe what we are working with. I have used the lxml parser to parse the page into BeautifulSoup Object.
page = urlopen(r'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050')

In [20]:
bs_obj = BeautifulSoup(page, 'lxml')

In [21]:
table = bs_obj.table

In [22]:
# Loosely based on the guidence. Creating and filling the table with the column headings and relevant data specified.

PostalCode=[]
Borough=[]
Neighborhood=[]

for tr in table.find_all('tr')[1:]:
    PostalCode.append(tr.find_all('td')[0].get_text())
    Borough.append(tr.find_all('td')[1].get_text())
    Neighborhood.append(tr.find_all('td')[2].get_text().strip('\n'))

In [23]:
# Cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

for i,j in enumerate(Neighborhood):
    if j=='Not assigned':
        if Borough[i]=='Not assigned':
            Neighborhood[i]=Borough[i]
        else:
            Neighborhood[i]=Borough[i]

In [24]:
df = pd.DataFrame({
        'PostalCode':PostalCode,
        'Borough':Borough,
        'Neighborhood':Neighborhood
    })

In [25]:
# Take a peak at what we have so far

df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [26]:
# As per the instructions, we now need to remove the cells that has a Borough 'Not assigned' and allow more than 1 neighbourhood

In [27]:
df['Borough'].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64

In [28]:
df = df[df.Borough!='Not assigned']

In [29]:
df_new = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [30]:
# As requested in instructions

df_new.shape

(103, 3)

In [31]:
# Showing table of the sake of completion
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [32]:
# Thank you for reading!


## Part 2 - Finding the Coordinates

In [33]:
# Using the data frame from above - for anyone struggling to make this work - upload the csv file in WS or skills network lab

df_coordinates = pd.read_csv(r'Geospatial_Coordinates.csv')

In [34]:
# check shape to see if it remains the same
df_coordinates.shape

(103, 3)

In [35]:
df_coordinates.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [36]:
# check columns
df_coordinates.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [37]:
#creation of new data frame
df_data = pd.merge(df_new, df_coordinates, on='PostalCode')

In [38]:
df_data.shape



(103, 5)

In [39]:
# showing the new data frame!
df_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [40]:
# Thank you for reading!


## Part 3: Exploring and attempting to cluster Toronto Neighbourhoods

In [41]:
# Need a library I didnt already import...
import folium

In [42]:
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [43]:

# Look back at the lab for how to do this.
for lat, lng, borough, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Borough'], df_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [44]:
# The instructions are only asking us to consider toronto, so we make a new data frame

toronto = df_data.loc[df_data['Borough'].str.contains('Toronto')].reset_index()

In [45]:
#  here we show the new data frame to check the above code worked
toronto

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [46]:
#Repeat the same code as above now that we have eliminated all boroughs out of Toronto 

In [47]:
latitude = 43.6543 
longitude = -79.3860
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [48]:
#  Lets explore Cabbagetown (loc 11)

In [49]:
toronto.loc[11, 'Neighborhood']

'Cabbagetown, St. James Town'

In [50]:
# Lets fetch the coordinates 
neighborhood_latitude = toronto.loc[11, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto.loc[11, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto.loc[11, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Cabbagetown, St. James Town are 43.667967, -79.3676753.


In [51]:
CLIENT_ID = 'Z3TCOEIVOMUNZ50QS2NU2SP15F2P25O12YPOH5KTMMWEIUP3' # your Foursquare ID
CLIENT_SECRET = '1WND5FSW1NOV05I2FWZC4V5JKJILLNQKYXDEZRQ3QQ3KLFDI' # your Foursquare Secret
ACCESS_TOKEN = 'SLPQYH33FUXYKTAXLIXLINYVI5SMCUUIBJ02VWMJVSWNRUIW' # your FourSquare Access Token
VERSION = '20210419'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: Z3TCOEIVOMUNZ50QS2NU2SP15F2P25O12YPOH5KTMMWEIUP3
CLIENT_SECRET:1WND5FSW1NOV05I2FWZC4V5JKJILLNQKYXDEZRQ3QQ3KLFDI


In [52]:
radius=200
LIMIT=10
VERSION='20190915'
import requests
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=Z3TCOEIVOMUNZ50QS2NU2SP15F2P25O12YPOH5KTMMWEIUP3&client_secret=1WND5FSW1NOV05I2FWZC4V5JKJILLNQKYXDEZRQ3QQ3KLFDI&v=20190915&ll=43.667967,-79.3676753&radius=200&limit=10'

In [53]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607dc3026bda057e40f21c71'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Cabbagetown',
  'headerFullLocation': 'Cabbagetown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 43.6697670018, 'lng': -79.36519153306388},
   'sw': {'lat': 43.6661669982, 'lng': -79.37015906693613}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e4e7aa06365e1419d021044',
       'name': "F'Amelia",
       'location': {'address': '12 Amelia St',
        'crossStreet': 'Parliament St',
        'lat': 43.66753590663226,
        'lng': -79.36861331485827,
        'labeledLatLngs': [{'label': 'display',


In [54]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [55]:
from pandas.io.json import json_normalize 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,F'Amelia,Italian Restaurant,43.667536,-79.368613
1,Cranberries,Diner,43.667843,-79.369407
2,Murgatroid,Restaurant,43.667381,-79.369311
3,Merryberry Cafe + Bistro,Café,43.666630,-79.368792
4,Cabbagetown Brew,Café,43.666923,-79.369289
5,Butter Chicken Factory,Indian Restaurant,43.667072,-79.369184
6,Absolute Bakery & Café,Bakery,43.667469,-79.369277
7,Toronto Dance Theatre,General Entertainment,43.666232,-79.367075
8,Tim Hortons,Coffee Shop,43.667169,-79.368849
9,The Beer Store,Beer Store,43.666408,-79.368833


In [56]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.
